In [1]:
from classes import load_model
import torch
import cv2
import numpy as np
import json

from torchvision import transforms

In [2]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
crop_size = 480

input_size = 160
num_frames = 16 # get 1 drop 1
threshold = 0.5
transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.CenterCrop((crop_size,crop_size)),
        # transforms.ColorJitter(brightness=(0.6,0.9),
        #                           contrast=(0.6,0.9),
        #                           hue=(-0.5,0.5),
        #                           saturation=(0.7,0.9)),
        # transforms.RandomRotation(3,expand=True),
        transforms.Resize((input_size,input_size)),
        transforms.ToTensor(),
        # transforms.Normalize(input_mean, input_std)
    ])

with open("label_list.json") as js:
    actions = list(json.load(js).values())
print(actions)

model = load_model()

['an', 'ban', 'ban dem', 'ban ngay', 'bo', 'cam on', 'choi', 'cuoi', 'di', 'di hoc', 'khoc', 'lam viec', 'me', 'moi ngay', 'sach', 'toi', 'viet', 'xem', 'xin chao', 'xin loi']


In [3]:
def format_frame(frame):
    frame = torch.tensor(frame,dtype=torch.float32)
    frame = frame.permute(2, 0, 1)
    frame = transform(frame)
    return frame

In [6]:
n = 0
drop = False
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if drop:
        drop = False
        continue
    else:
        drop = True
        image = cv2.flip(frame,1) # this is for showing
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = format_frame(frame)
        if n<16:
            if n == 0:
                sequence = frame.unsqueeze(0)
            else:
                sequence = torch.cat((sequence, frame.unsqueeze(0)), dim=0)
            n += 1
            cv2.imshow("FullScreen", image)
        else:
            sequence = sequence.permute(1,0,2,3)
            output = model(sequence.expand(1,3,16,160,160))
            output = output.detach().numpy()
            sequence = None
            label = actions[np.argmax(output)]
            n = 0
            cv2.putText(image, f'Predicted: {label}', (80,150), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
            cv2.imshow("FullScreen", image)
            if cv2.waitKey(2000) & 0xFF == ord('q'):
                break
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 